In [39]:
import pandas as pd
import numpy as np
import my_def

<h1>데이터확인</h1>

In [40]:
df = pd.read_csv('Merged_Sales_data_v2.5.csv') # 월매출액 / 점포수

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374581 entries, 0 to 374580
Data columns (total 13 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   년분기        374581 non-null  int64 
 1   행정동        374581 non-null  object
 2   업종명        374581 non-null  object
 3   월매출(점포)    374581 non-null  int64 
 4   인구수        374581 non-null  int64 
 5   지역생활인구     374581 non-null  int64 
 6   장기외국인      374581 non-null  int64 
 7   단기외국인      374581 non-null  int64 
 8   주차장면적(면)   374581 non-null  int64 
 9   주차장개수(개소)  374581 non-null  int64 
 10  학교수        374581 non-null  int64 
 11  학생수        374581 non-null  int64 
 12  버스정류장수     374581 non-null  int64 
dtypes: int64(11), object(2)
memory usage: 37.2+ MB


In [42]:
df = my_def.reduce_memory_usage(df) # 메모리 사용량 감소를 위한 Dtype 변환

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374581 entries, 0 to 374580
Data columns (total 13 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   년분기        374581 non-null  int16 
 1   행정동        374581 non-null  object
 2   업종명        374581 non-null  object
 3   월매출(점포)    374581 non-null  int64 
 4   인구수        374581 non-null  int32 
 5   지역생활인구     374581 non-null  int32 
 6   장기외국인      374581 non-null  int32 
 7   단기외국인      374581 non-null  int32 
 8   주차장면적(면)   374581 non-null  int32 
 9   주차장개수(개소)  374581 non-null  int16 
 10  학교수        374581 non-null  int16 
 11  학생수        374581 non-null  int16 
 12  버스정류장수     374581 non-null  int16 
dtypes: int16(5), int32(5), int64(1), object(2)
memory usage: 19.3+ MB
None


<h1>전체업종 선형회귀 모델</h1>
Polynomial Features & Standard Scaler 사용 안함

In [43]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [44]:
# '행정동' 컬럼 원핫인코딩 수행
df = pd.get_dummies(df, columns=['행정동'], drop_first=True)

# 피처와 타겟 설정
features = ['년분기', '인구수', '지역생활인구', '장기외국인', '단기외국인', '주차장면적(면)', '주차장개수(개소)', '학교수', '학생수', '버스정류장수'] + [col for col in df.columns if col.startswith('행정동_')]
target = '월매출(점포)'

X = df[features]
y = df[target]

# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 선형 회귀 모델 학습 및 예측
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [45]:
# 모델 평가 호출 선형회귀모델
my_def.evaluate_model(model, X_train, X_test, y_train, y_test)

{'train_mse': np.float64(8156353667657603.0),
 'test_mse': np.float64(8326205856834486.0),
 'train_rmse': np.float64(90312533.28),
 'test_rmse': np.float64(91248045.77),
 'train_r2': 0.05,
 'test_r2': 0.05,
 'train_mae': np.float64(44296374.69),
 'test_mae': np.float64(44513597.43),
 'train_mae_ratio': np.float64(99.57),
 'test_mae_ratio': np.float64(100.02)}

<h1>개별업종 선형모델 평가</h1>
Polynomial Features & Standard Scaler 사용 안함

In [46]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import pandas as pd

# 파일 로드
file_path = 'Merged_Sales_data_v2.5.csv'
df = pd.read_csv(file_path)

# 업종명 고유값 추출
unique_categories = df["업종명"].unique()

# 결과를 저장할 딕셔너리 생성
results = {}

# 각 업종별로 데이터를 나누어 모델 학습 및 예측 수행
for category in unique_categories:
    # 업종별 데이터 필터링
    df_category = df[df['업종명'] == category]
    
    # 원핫인코딩 수행
    df_category = pd.get_dummies(df_category, columns=['행정동'], drop_first=True)
    
    # 피처와 타겟 설정
    features = ['년분기', '인구수', '지역생활인구', '장기외국인', '단기외국인', '주차장면적(면)', '주차장개수(개소)', '학교수', '학생수', '버스정류장수'] + [col for col in df_category.columns if col.startswith('행정동_')]
    target = '월매출(점포)'
    
    # 피처와 타겟 데이터 분리
    X = df_category[features]
    y = df_category[target]
    
    # 학습 데이터와 테스트 데이터 분리
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # 선형 회귀 모델 학습
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # 성능 평가 호출
    evaluation_result = my_def.evaluate_model(model, X_train, X_test, y_train, y_test)
    
    # 결과 저장
    if evaluation_result is not None:
        results[category] = evaluation_result

# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df = results_df.apply(pd.to_numeric)

# test_mae_ratio의 최고, 최저, 평균 추가
test_mae_ratio_max = results_df['test_mae_ratio'].max()
test_mae_ratio_min = results_df['test_mae_ratio'].min()
test_mae_ratio_mean = results_df['test_mae_ratio'].mean()

# 모든 행에 공통으로 최고, 최저, 평균 추가
test_mae_summary = {
    'test_mae_ratio_max': test_mae_ratio_max,
    'test_mae_ratio_min': test_mae_ratio_min,
    'test_mae_ratio_mean': test_mae_ratio_mean
}

for key, value in test_mae_summary.items():
    results_df[key] = value

# 결과를 소수점 2자리로 포맷팅하여 출력
pd.options.display.float_format = '{:.2f}'.format
print(results_df)


                  train_mse                       test_mse   train_rmse  \
미곡판매    5933382120252120.00 187486351853406446541602816.00  77028450.07   
분식전문점     66117949502714.05              50457373774981.29   8131294.45   
편의점      565354458427281.50             671657762850320.62  23777183.57   
한식음식점     83970274937360.14             119656390845344.66   9163529.61   
의약품     3297095806791203.00            4385685765224285.00  57420343.14   
...                     ...                            ...          ...   
수산물판매   1714562520747046.50            1813624527369598.50  41407276.18   
고시원      140393217360410.59             271130506892865.09  11848764.38   
완구     13933979911827928.00            7829912655514360.00 118042280.19   
부동산중개업     2071525093996.14               4352410749408.58   1439279.37   
가방       398541670760167.50             604090853835333.75  19963508.48   

               test_rmse  train_r2        test_r2   train_mae  \
미곡판매   13692565568709.41      0.88

In [47]:
print(results_df.loc["한식음식점"])

train_mse              83970274937360.14
test_mse              119656390845344.66
train_rmse                    9163529.61
test_rmse                    10938756.37
train_r2                            0.84
test_r2                             0.80
train_mae                     6645764.80
test_mae                      7584013.79
train_mae_ratio                    15.59
test_mae_ratio                     16.87
test_mae_ratio_max            2660933.49
test_mae_ratio_min                 15.42
test_mae_ratio_mean             82514.08
Name: 한식음식점, dtype: float64


In [48]:
# test_mae_ratio 값이 큰 순으로 상위 10개 추출
top_10_test_mae = results_df['test_mae_ratio'].nlargest(10)

# test_mae_ratio 값이 작은 순으로 하위 10개 추출
tail_10_test_mae = results_df['test_mae_ratio'].nsmallest(10)

# 결과 출력
print("Top 10 test_mae_ratio:")
print(top_10_test_mae)
print("*" * 50)
print("Bottom 10 test_mae_ratio:")
print(tail_10_test_mae)

Top 10 test_mae_ratio:
애완동물      2660933.49
미곡판매      1517325.70
PC방        634784.74
운동/경기용품    173452.75
화초         163137.50
여관          46922.99
핸드폰            61.39
전자상거래업         60.52
부동산중개업         53.16
인테리어           52.72
Name: test_mae_ratio, dtype: float64
**************************************************
Bottom 10 test_mae_ratio:
편의점     15.42
미용실     15.97
한식음식점   16.87
치과의원    17.54
안경      17.75
일반의원    18.15
의약품     18.25
자동차수리   19.06
한의원     19.19
슈퍼마켓    20.09
Name: test_mae_ratio, dtype: float64


<h1>Standard Scaler 적용</h1>

In [49]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import pandas as pd

# 파일 로드
file_path = 'Merged_Sales_data_v2.5.csv'
df = pd.read_csv(file_path)

# 업종명 고유값 추출
unique_categories = df["업종명"].unique()

# 결과를 저장할 딕셔너리 생성
results = {}

# 각 업종별로 데이터를 나누어 모델 학습 및 예측 수행
for category in unique_categories:
    # 업종별 데이터 필터링
    df_category = df[df['업종명'] == category]
    
    # 원핫인코딩 수행
    df_category = pd.get_dummies(df_category, columns=['행정동'], drop_first=True)
    
    # 피처와 타겟 설정
    features = ['년분기', '인구수', '지역생활인구', '장기외국인', '단기외국인', '주차장면적(면)', '주차장개수(개소)', '학교수', '학생수', '버스정류장수'] + [col for col in df_category.columns if col.startswith('행정동_')]
    target = '월매출(점포)'
    
    # 피처와 타겟 데이터 분리
    X = df_category[features]
    y = df_category[target]
    
    # Standard Scaler 적용
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # 학습 데이터와 테스트 데이터 분리
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    
    # 선형 회귀 모델 학습
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # 성능 평가 호출
    evaluation_result = my_def.evaluate_model(model, X_train, X_test, y_train, y_test)
    
    # 결과 저장
    if evaluation_result is not None:
        results[category] = evaluation_result

# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df = results_df.apply(pd.to_numeric)

# test_mae_ratio의 최고, 최저, 평균 추가
test_mae_ratio_max = results_df['test_mae_ratio'].max()
test_mae_ratio_min = results_df['test_mae_ratio'].min()
test_mae_ratio_mean = results_df['test_mae_ratio'].mean()

# 모든 행에 공통으로 최고, 최저, 평균 추가
test_mae_summary = {
    'test_mae_ratio_max': test_mae_ratio_max,
    'test_mae_ratio_min': test_mae_ratio_min,
    'test_mae_ratio_mean': test_mae_ratio_mean
}

for key, value in test_mae_summary.items():
    results_df[key] = value

# 결과를 소수점 2자리로 포맷팅하여 출력
pd.options.display.float_format = '{:.2f}'.format
print(results_df)

                  train_mse                                      test_mse  \
미곡판매    5933382120251782.00                        1782956849750439680.00   
분식전문점     66117949502714.11                             50457373772763.79   
편의점      565354458427282.12                            671657762835445.88   
한식음식점     83970274937359.55                            119656391108593.31   
의약품     3297095806791198.50                           4385685765492504.50   
...                     ...                                           ...   
수산물판매   1714562520747042.25                           1813624526605816.00   
고시원      140393217360410.56                            271130506892844.81   
완구     13933979911827978.00                           7829912655514275.00   
부동산중개업     2071525093996.14                              3287385006331.12   
가방       398564524061700.19 114829281154428379706766150206943976751104.00   

         train_rmse                test_rmse  train_r2  \
미곡판매    77028450.

In [50]:
print(results_df.loc["한식음식점"])

train_mse              83970274937359.55
test_mse              119656391108593.31
train_rmse                    9163529.61
test_rmse                    10938756.38
train_r2                            0.84
test_r2                             0.80
train_mae                     6645764.80
test_mae                      7584013.79
train_mae_ratio                    15.59
test_mae_ratio                     16.87
test_mae_ratio_max    252965611972586.56
test_mae_ratio_min                 15.42
test_mae_ratio_mean    12158593732729.86
Name: 한식음식점, dtype: float64


In [51]:
# test_mae_ratio 값이 큰 순으로 상위 10개 추출
top_10_test_mae = results_df['test_mae_ratio'].nlargest(10)

# test_mae_ratio 값이 작은 순으로 하위 10개 추출
tail_10_test_mae = results_df['test_mae_ratio'].nsmallest(10)

# 결과 출력
print("Top 10 test_mae_ratio:")
print(top_10_test_mae)
print("*" * 50)
print("Bottom 10 test_mae_ratio:")
print(tail_10_test_mae)

Top 10 test_mae_ratio:
시계및귀금속         252965611972586.56
여관             114444817399898.45
자전거 및 기타운송장비   106151680026948.19
가방              66407262943700.05
전자상거래업          49897389501089.62
운동/경기용품         47042446753156.55
인테리어            36197484551505.90
컴퓨터및주변장치판매      28534615058272.08
애완동물            28074516655686.62
가전제품수리          20195298939560.32
Name: test_mae_ratio, dtype: float64
**************************************************
Bottom 10 test_mae_ratio:
편의점     15.42
미용실     15.97
한식음식점   16.87
치과의원    17.54
안경      17.75
일반의원    18.15
의약품     18.25
자동차수리   19.06
한의원     19.19
슈퍼마켓    20.09
Name: test_mae_ratio, dtype: float64


<h1>Min-Max Scaler 적용</h1>

In [52]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# 파일 로드
file_path = 'Merged_Sales_data_v2.5.csv'
df = pd.read_csv(file_path)

# 업종명 고유값 추출
unique_categories = df["업종명"].unique()

# 결과를 저장할 딕셔너리 생성
results = {}

# 각 업종별로 데이터를 나누어 모델 학습 및 예측 수행
for category in unique_categories:
    # 업종별 데이터 필터링
    df_category = df[df['업종명'] == category]
    
    # 원핫인코딩 수행
    df_category = pd.get_dummies(df_category, columns=['행정동'], drop_first=True)
    
    # 피처와 타겟 설정
    features = ['년분기', '인구수', '지역생활인구', '장기외국인', '단기외국인', '주차장면적(면)', '주차장개수(개소)', '학교수', '학생수', '버스정류장수'] + [col for col in df_category.columns if col.startswith('행정동_')]
    target = '월매출(점포)'
    
    # 피처와 타겟 데이터 분리
    X = df_category[features]
    y = df_category[target]
    
    # Min-Max Scaler 적용
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    
    # 학습 데이터와 테스트 데이터 분리
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    
    # 선형 회귀 모델 학습
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # 성능 평가 호출
    evaluation_result = my_def.evaluate_model(model, X_train, X_test, y_train, y_test)
    
    # 결과 저장
    if evaluation_result is not None:
        results[category] = evaluation_result

# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df = results_df.apply(pd.to_numeric)

# test_mae_ratio의 최고, 최저, 평균 추가
test_mae_ratio_max = results_df['test_mae_ratio'].max()
test_mae_ratio_min = results_df['test_mae_ratio'].min()
test_mae_ratio_mean = results_df['test_mae_ratio'].mean()

# 모든 행에 공통으로 최고, 최저, 평균 추가
test_mae_summary = {
    'test_mae_ratio_max': test_mae_ratio_max,
    'test_mae_ratio_min': test_mae_ratio_min,
    'test_mae_ratio_mean': test_mae_ratio_mean
}

for key, value in test_mae_summary.items():
    results_df[key] = value

# 결과를 소수점 2자리로 포맷팅하여 출력
pd.options.display.float_format = '{:.2f}'.format
print(results_df)

                  train_mse                                     test_mse  \
미곡판매    5933382120251781.00                         42079939143096080.00   
분식전문점     66117949502714.12                            50457373772763.81   
편의점      565354458427281.88                           671657762835448.88   
한식음식점     83970274937359.58                           119656391108595.05   
의약품     3297095806791197.50                          4385685765492494.50   
...                     ...                                          ...   
수산물판매   1714562520747042.25                          1813624526605815.00   
고시원      140393217360410.44                           271130506892843.62   
완구     13933979911827952.00                          7829912655514304.00   
부동산중개업     2071525093996.14                             4352410749408.44   
가방       398571033260535.81 15159331772672106581186255680818158501888.00   

         train_rmse                test_rmse  train_r2  \
미곡판매    77028450.07          

In [53]:
print(results_df.loc["한식음식점"])

train_mse              83970274937359.58
test_mse              119656391108595.05
train_rmse                    9163529.61
test_rmse                    10938756.38
train_r2                            0.84
test_r2                             0.80
train_mae                     6645764.80
test_mae                      7584013.79
train_mae_ratio                    15.59
test_mae_ratio                     16.87
test_mae_ratio_max     32981049624083.46
test_mae_ratio_min                 15.42
test_mae_ratio_mean      895344939792.50
Name: 한식음식점, dtype: float64


In [54]:
# test_mae_ratio 값이 큰 순으로 상위 10개 추출
top_10_test_mae = results_df['test_mae_ratio'].nlargest(10)

# test_mae_ratio 값이 작은 순으로 하위 10개 추출
tail_10_test_mae = results_df['test_mae_ratio'].nsmallest(10)

# 결과 출력
print("Top 10 test_mae_ratio:")
print(top_10_test_mae)
print("*" * 50)
print("Bottom 10 test_mae_ratio:")
print(tail_10_test_mae)

Top 10 test_mae_ratio:
가방             32981049624083.46
시계및귀금속          4459450181306.03
애완동물            3787354343215.49
PC방             3169584746837.00
인테리어            2343634202026.49
여관              2063629998199.99
스포츠클럽           1731362971174.72
자전거 및 기타운송장비    1145772796062.68
섬유제품             996814506488.66
화초               800195690552.95
Name: test_mae_ratio, dtype: float64
**************************************************
Bottom 10 test_mae_ratio:
편의점     15.42
미용실     15.97
한식음식점   16.87
치과의원    17.54
안경      17.75
일반의원    18.15
의약품     18.25
자동차수리   19.06
한의원     19.19
슈퍼마켓    20.09
Name: test_mae_ratio, dtype: float64
